In [401]:
import numpy as np
import pandas as pd
import datetime

# Load your 1-minute trading data (replace 'your_data.csv' with your dataset)
rowdata = pd.read_csv('/Users/yuding/Downloads/NASDAQ_QQQ_1.csv')
# Convert time from UnixTimeStamp to regular time
rowdata["time1"] = [datetime.datetime.fromtimestamp(time) for time in rowdata['time']]
# Split data day by day
rowdata["DateDay"] = [str(time).split(" ")[0] for time in rowdata["time1"]]

class Data:
    def __init__(self, df):
        self.startDate = df["DateDay"][0]
        self.endData = df["DateDay"][len(df["DateDay"]) - 1]
        self.grouped = df.groupby('DateDay')
        self.size = self.grouped.size()
        self.numDays = len(self.grouped)
        self.dates = self.size.keys()
        self.lengths = self.size.values
        self.index = 0
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.index < self.numDays:
            result = self.grouped.get_group(self.dates[self.index])
            self.index += 1
            return result
        else:
            raise StopIteration
            
    def __getitem__(self, key):
        # Handle single indices or slices
        if isinstance(key, int):
            return self.grouped.get_group(dates[key])
        elif isinstance(key, slice):
            start, stop, step = key.start, key.stop, key.step
            return self.grouped[dates[start:stop:step]]
        else:
            raise TypeError("Invalid key type; only integers or slices are allowed.")
        
    def __len__(self):
        return self.numDays


In [403]:
rowdata['close'].max(), rowdata['close'].min()

(384.84, 354.92)

In [404]:
datas = Data(rowdata)

In [407]:
import numpy as np
import pandas as pd

# Create a simple toy stock trading environment
class StockTradingEnvironment:
    def __init__(self, data):
        self.data = data
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = 1000.0  # Initial account balance
        self.shares_held = 0
        self.stock_price = self.data[self.current_step]
        return [self.balance, self.shares_held, self.stock_price]

    def step(self, action):
        self.current_step += 1
        done = False

        if self.current_step >= len(self.data) - 1:
            done = True

        if action == 0:  # Buy
            if self.balance >= self.stock_price:
                self.shares_held += 1
                self.balance -= self.stock_price
        elif action == 1:  # Hold
            pass
        elif action == 2:  # Sell
            if self.shares_held > 0:
                self.shares_held -= 1
                self.balance += self.stock_price

        self.stock_price = self.data[self.current_step]

        # Calculate the reward (e.g., change in account balance)
        reward = self.balance - 1000.0  # Starting balance

        return [self.balance, self.shares_held, self.stock_price], reward, done

# Load historical stock price data (replace with your own dataset)
# data = pd.DataFrame({'price': [100, 105, 110, 115, 120, 125, 130, 125, 120, 115]})

# Create the stock trading environment


# Q-learning hyperparameters
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.2
num_episodes = 10

# Q-table initialization (state-action space)
num_actions = 3  # Buy, Hold, Sell
q_table = np.zeros((40000, num_actions))

In [408]:
state = env.reset()
state

[1000.0, 0, 376.4]

In [ ]:
# Q-learning algorithm
for episode in range(num_episodes):
    print(f"episode: {episode}")
    datas = Data(rowdata)
    for data in datas:
        print(data)
        data = list(data['close'])
        env = StockTradingEnvironment(data)
        state = env.reset()
        done = False
        while not done:
            if np.random.rand() < epsilon:
                action = np.random.choice(num_actions)
            else:
                action = np.argmax(q_table[int(100 * state[2]), :])
            print(f"episode: {episode}, action: {action}, state: {state}")
            next_state, reward, done = env.step(action)

            q_table[int(100 * state[2]), action] = (1 - learning_rate) * q_table[int(100 * state[2]), action] + \
                                    learning_rate * (reward + discount_factor * np.max(q_table[int(100 * state[2]), :]))

            state = next_state

episode: 0
           time    open    high       low   close  Volume  Volume MA  \
0    1690790400  382.86  383.01  382.8600  383.01     894        NaN   
1    1690790460  382.99  382.99  382.9800  382.98     530        NaN   
2    1690790520  383.04  383.05  382.9300  382.93    1630        NaN   
3    1690790580  383.02  383.02  383.0200  383.02     100        NaN   
4    1690790640  383.03  383.03  383.0100  383.03    1971        NaN   
..          ...     ...     ...       ...     ...     ...        ...   
817  1690847640  383.80  383.80  383.8000  383.80     610    1083.45   
818  1690847700  383.78  383.82  383.7800  383.82     595    1058.30   
819  1690847760  383.78  383.78  383.7800  383.78     699     901.95   
820  1690847880  383.78  383.80  383.7800  383.80     500     918.20   
821  1690847940  383.82  383.83  383.8096  383.82    4295    1122.45   

                  time1     DateDay  
0   2023-07-31 04:00:00  2023-07-31  
1   2023-07-31 04:01:00  2023-07-31  
2   2023-0

episode: 0, action: 1, state: [995.9996000000003, 0, 368.16]
episode: 0, action: 2, state: [995.9996000000003, 0, 368.19]
episode: 0, action: 2, state: [995.9996000000003, 0, 368.23]
episode: 0, action: 2, state: [995.9996000000003, 0, 368.22]
episode: 0, action: 2, state: [995.9996000000003, 0, 368.23]
episode: 0, action: 0, state: [995.9996000000003, 0, 368.21]
episode: 0, action: 2, state: [627.7896000000003, 1, 368.135]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.25]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.2]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.4]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.455]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.46]
episode: 0, action: 0, state: [995.9246000000003, 0, 368.47]
episode: 0, action: 2, state: [627.4546000000003, 1, 368.47]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.45]
episode: 0, action: 2, state: [995.9246000000003, 0, 368.36]
episode: 0, action: 2, s

In [393]:
# Test the learned policy
state = env.reset()
done = False

while not done:
    action = np.argmax(q_table[int(100 * state[2]), :])
    state, _, done = env.step(action)
    print(f"state: {state}, action: {action}")

print(f"Final balance: ${state[0]:.2f}")

state: [1000.0, 0, 376.57], action: 2
state: [1000.0, 0, 376.71], action: 2
state: [1000.0, 0, 376.7], action: 2
state: [1000.0, 0, 376.72], action: 2
state: [1000.0, 0, 376.81], action: 2
state: [1000.0, 0, 376.95], action: 2
state: [1000.0, 0, 377.1], action: 2
state: [1000.0, 0, 377.05], action: 2
state: [1000.0, 0, 377.04], action: 2
state: [1000.0, 0, 377.0], action: 2
state: [1000.0, 0, 377.0], action: 2
state: [1000.0, 0, 376.95], action: 2
state: [1000.0, 0, 376.98], action: 2
state: [1000.0, 0, 376.95], action: 2
state: [1000.0, 0, 376.92], action: 2
state: [1000.0, 0, 376.9], action: 2
state: [1000.0, 0, 376.7], action: 2
state: [1000.0, 0, 376.64], action: 2
state: [1000.0, 0, 376.67], action: 2
state: [1000.0, 0, 376.64], action: 2
state: [1000.0, 0, 376.65], action: 2
state: [1000.0, 0, 376.76], action: 2
state: [1000.0, 0, 376.66], action: 2
state: [1000.0, 0, 376.74], action: 2
state: [1000.0, 0, 376.74], action: 2
state: [1000.0, 0, 376.66], action: 2
state: [1000.0, 0,